In [2]:
import os
import pandas as pd

In [3]:
group = 1
inPath = f'/home/sdodl001/Desktop/DNA_Methylation_Scripts/cpg_util_scripts/data/kmeans/uncert_gve_direction/{group}/pred200_merged/'
files = os.listdir(inPath)
files.sort()
len(files)

148

In [6]:
# Initialize an empty set to store unique name values
unique_names = set()

# Track progress
total_files = len(files)
processed = 0

# Process files in batches to avoid memory issues
for file in files:
    try:
        # Only load the 'name' column to minimize memory usage
        data = pd.read_parquet(inPath + file, columns=['name'])
        
        # Update the set with new unique values
        unique_names.update(data['name'])
        
        # Update and display progress
        processed += 1
        if processed % 10 == 0 or processed == total_files:
            print(f"Processed {processed}/{total_files} files. Current unique names: {len(unique_names)}")
            
    except Exception as e:
        print(f"Error processing {file}: {str(e)}")

print(f"\nTotal unique names collected: {len(unique_names)}")
print(f"Memory used by unique_names set: {sum(len(name) for name in unique_names) / (1024*1024):.2f} MB")

# Preview some unique names (first 5)
list(unique_names)[:5] if unique_names else "No unique names found"

Processed 10/148 files. Current unique names: 3645640
Processed 20/148 files. Current unique names: 4195255
Processed 30/148 files. Current unique names: 4554064
Processed 40/148 files. Current unique names: 4695465
Processed 50/148 files. Current unique names: 4931860
Processed 60/148 files. Current unique names: 5099470
Processed 70/148 files. Current unique names: 5138015
Processed 80/148 files. Current unique names: 5149679
Processed 90/148 files. Current unique names: 5257657
Processed 100/148 files. Current unique names: 7958108
Processed 110/148 files. Current unique names: 8837202
Processed 120/148 files. Current unique names: 9015023
Processed 130/148 files. Current unique names: 9120615
Processed 140/148 files. Current unique names: 10246896
Processed 148/148 files. Current unique names: 10358075

Total unique names collected: 10358075
Memory used by unique_names set: 114.34 MB


['rs567865040', 'rs1426319946', 'rs927581744', 'rs1038629972', 'rs928388916']